<a href="https://colab.research.google.com/github/L0ki2026/MarketHarvest/blob/main/Copy_of_evaluationMayank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.6/913.6 kB 7.1 MB/s eta 0:00:00


## 0. Upload Model

In [ ]:
from google.colab import files

# Prompt the user to upload the file
model = files.upload()

# Access the uploaded file
for filename, file_content in model.items():
    print(f"Uploaded file: {filename}, size: {len(file_content)} bytes")

Saving best.pt to best.pt
Uploaded file: best.pt, size: 40462885 bytes


## 1. Create Data

In [ ]:
# Download Data from the web
!wget -O cit_det.zip "https://mavmatrix.uta.edu/context/cse_datasets/article/1000/type/native/viewcontent"
!wget -O apple_data.tar.gz "https://conservancy.umn.edu/bitstreams/3ef26f04-6467-469b-9857-f443ffa1bb61/download"

# Unzip the data
!unzip /content/cit_det.zip
!tar -xvzf /content/apple_data.tar.gz
!unzip /content/CitDet-test.zip
!unzip /content/CitDet-train.zip

# Remove zip files
!rm /content/CitDet-test.zip /content/CitDet-train.zip /content/cit_det.zip /content/MANIFEST.TXT
!rm /content/apple_data.tar.gz

# Create file structure
!mkdir citrus_data
!mv /content/test citrus_data
!mv /content/train citrus_data
!mv detection apple_data
!mv /content/citrus_data/test/images/* /content/citrus_data/train/images/
!mv /content/citrus_data/test/masks/* /content/citrus_data/train/masks/

--2025-01-27 21:31:27--  https://mavmatrix.uta.edu/context/cse_datasets/article/1000/type/native/viewcontent
Resolving mavmatrix.uta.edu (mavmatrix.uta.edu)... 50.18.241.247, 13.57.92.51
Connecting to mavmatrix.uta.edu (mavmatrix.uta.edu)|50.18.241.247|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://mavmatrix.uta.edu/cgi/viewcontent.cgi?params=/context/cse_datasets/article/1000/type/native/&path_info= [following]
--2025-01-27 21:31:27--  https://mavmatrix.uta.edu/cgi/viewcontent.cgi?params=/context/cse_datasets/article/1000/type/native/&path_info=
Reusing existing connection to mavmatrix.uta.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 1103158596 (1.0G) [application/zip]
Saving to: ‘cit_det.zip’

cit_det.zip         100%[===================>]   1.03G  80.5MB/s    in 14s     

2025-01-27 21:31:41 (75.0 MB/s) - ‘cit_det.zip’ saved [1103158596/1103158596]

--2025-01-27 21:31:42--  https://conservancy.umn.edu/bitstreams/

In [ ]:
import os
import torch
from torchvision.io import read_image
from torchvision.ops import masks_to_boxes
import numpy as np

# Paths
img_folder = r"/content/apple_data/train/images"
mask_folder = r"/content/apple_data/train/masks"
output_folder = r"/content/apple_data/train/masks"


# Iterate through all images and masks
for mask_file in os.listdir(mask_folder):
    # Ensure it's a valid image file
    if not mask_file.endswith(('.png', '.jpg', '.jpeg', '.tiff')):
        continue

    # Paths for the current image and mask
    mask_path = os.path.join(mask_folder, mask_file)
    img_path = os.path.join(img_folder, mask_file)

    # Read the mask
    mask = read_image(mask_path)

    # Get unique object IDs (skipping background)
    obj_ids = torch.unique(mask)
    obj_ids = obj_ids[1:]

    # Create binary masks for each object
    masks = mask == obj_ids[:, None, None]

    # Generate bounding boxes
    boxes = masks_to_boxes(masks)

    # Normalize boxes for YOLO format
    img_height, img_width = mask.shape[1:]  # H, W
    yolo_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        yolo_boxes.append([0, x_center.item(), y_center.item(), width.item(), height.item()])

    # Save annotations to a YOLO format file
    yolo_file = os.path.join(output_folder, os.path.splitext(mask_file)[0] + ".txt")
    with open(yolo_file, "w") as f:
        for box in yolo_boxes:
            f.write(" ".join(map(str, box)) + "\n")

print(f"Annotations saved in YOLO format to: {output_folder}")


Annotations saved in YOLO format to: /content/apple_data/train/masks


In [ ]:
# for citrus
import cv2
from skimage.measure import label, regionprops

# Paths
mask_folder = r"/content/citrus_data/train/masks"
output_folder = r"/content/citrus_data/train/masks"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process each mask
for mask_file in os.listdir(mask_folder):
    if not mask_file.endswith(('.png', '.jpg', '.jpeg', '.tiff')):
        continue

    # Read the mask as a grayscale image
    mask_path = os.path.join(mask_folder, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Apply connected component labeling
    labeled_mask = label(mask > 1)  # All greater than 1 pixels are considered part of a component

    # Extract bounding boxes from labeled regions
    img_height, img_width = mask.shape
    yolo_boxes = []

    for region in regionprops(labeled_mask):
        # Get bounding box (min_row, min_col, max_row, max_col)
        min_row, min_col, max_row, max_col = region.bbox

        # Convert to YOLO format (x_center, y_center, width, height)
        x_center = ((min_col + max_col) / 2) / img_width
        y_center = ((min_row + max_row) / 2) / img_height
        width = (max_col - min_col) / img_width
        height = (max_row - min_row) / img_height

        yolo_boxes.append([1, x_center, y_center, width, height])  # Class ID is set to 1

    # Save annotations in YOLO format
    yolo_file = os.path.join(output_folder, os.path.splitext(mask_file)[0] + ".txt")
    with open(yolo_file, "w") as f:
        for box in yolo_boxes:
            f.write(" ".join(map(str, box)) + "\n")

print(f"Annotations saved in YOLO format to: {output_folder}")


Annotations saved in YOLO format to: /content/citrus_data/train/masks


In [ ]:
# Remove segmentation masks
!rm -rf /content/citrus_data/train/masks/*.png
!rm -rf /content/apple_data/train/masks/*.png

# Create final file structure
!mkdir -p combined_data/images/train combined_data/images/val combined_data/labels/train combined_data/labels/val

# Move images and annotations into the correct folder
!mv /content/citrus_data/train/images/* /content/combined_data/images/train
!mv /content/apple_data/train/images/* /content/combined_data/images/train

!mv /content/apple_data/train/masks/* /content/combined_data/labels/train/
!mv /content/citrus_data/train/masks/* /content/combined_data/labels/train/

# Remove old files
!rm -rf /content/citrus_data /content/apple_data

In [ ]:
# Example YAML content
yaml_content = """
path: /content/combined_data
train: images/train
val: images/val

nc: 2
# Classes (80 COCO classes)
names: ['apple', 'citrus']
"""

# Write to a YAML file
with open("config.yaml", "w") as file:
    file.write(yaml_content)

print("YAML file created!")

YAML file created!


In [ ]:

import random
import shutil

# Set your dataset paths
images_path = r"/content/combined_data/images/train"
annotations_path = r"/content/combined_data/labels/train"

# Destination paths
train_images_dest = r"/content/combined_data/images/train"
val_images_dest = r"/content/combined_data/images/val"
train_annotations_dest = r"/content/combined_data/labels/train"
val_annotations_dest = r"/content/combined_data/labels/val"

# Create necessary folders
os.makedirs(train_images_dest, exist_ok=True)
os.makedirs(val_images_dest, exist_ok=True)
os.makedirs(train_annotations_dest, exist_ok=True)
os.makedirs(val_annotations_dest, exist_ok=True)

# Get the list of all images and shuffle them
images = [f for f in os.listdir(images_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(images)

# Split the data into 80% train and 20% val
split_ratio = 0.80
split_index = int(len(images) * split_ratio)

train_images = images[:split_index]
val_images = images[split_index:]

# Function to move files
def move_files(file_list, src_images_path, src_annotations_path, dest_images_path, dest_annotations_path):
    for image in file_list:
        # Move image
        shutil.move(os.path.join(src_images_path, image), os.path.join(dest_images_path, image))

        # Move corresponding annotation
        annotation_file = image.replace('.png', '.txt').replace('.jpg', '.txt').replace('.jpeg', '.txt')
        shutil.move(os.path.join(src_annotations_path, annotation_file), os.path.join(dest_annotations_path, annotation_file))

# Move train files
move_files(train_images, images_path, annotations_path, train_images_dest, train_annotations_dest)

# Move val files
move_files(val_images, images_path, annotations_path, val_images_dest, val_annotations_dest)

print("Train-Test split completed successfully!")


Train-Test split completed successfully!


## 2. Eval

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11m.pt")

metrics = model.val(data='/content/config.yaml', imgsz=320)

Ultralytics 8.3.68 🚀 Python-3.11.11 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/combined_data/labels/val.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:29<00:00,  1.84s/it]


                   all        250      12430        0.7      0.471      0.526      0.228
                 apple        132       5473      0.686      0.593      0.624      0.261
                citrus        118       6957      0.714      0.349      0.428      0.195
Speed: 0.4ms preprocess, 48.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/val3
